In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime
%matplotlib inline
import seaborn as sns
sns.set()
from matplotlib.backends.backend_pdf import PdfPages
#from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm
import matplotlib.colors as col
import matplotlib as mpl
import numpy as np #import for transposing
import math
import pylab as pl
import mpld3 as ml
import math
import import_ipynb
from sklearn.metrics import mean_squared_error,mean_absolute_error
from function_log import add_tdate,get_week,density_interp,extract_date,qtr_comp,IC_run_sim,new_compare,stoch_model,run_sim

from scipy.optimize import minimize

importing Jupyter notebook from function_log.ipynb


In [2]:
def find(condition):
    res, = np.nonzero(np.ravel(condition))
    return res

In [3]:
new_census = pd.read_csv('new_census.csv')
new_census['Date'] = pd.to_datetime(new_census['Date'])
new_census['ObsYear'] = new_census.Date.apply(lambda x: x.year)
new_census['ObsMonth'] = new_census.Date.apply(lambda x:x.month)

new_census.T = pd.to_numeric(new_census['T'],errors='coerce')
new_census.J = pd.to_numeric(new_census.J)
new_census.A = pd.to_numeric(new_census.A)
new_census.S = pd.to_numeric(new_census.S)

In [4]:
t_types = pd.read_csv('troop_types.csv')

rural_troops = list(t_types.loc[t_types['Type Code'] == 0]['Troop'])
rural = new_census.loc[new_census.Troop.isin(rural_troops)]

urb_troops = list(t_types.loc[t_types['Type Code'] == 1]['Troop'])
urban = new_census.loc[new_census.Troop.isin(urb_troops)]

In [5]:
from __future__ import print_function
import sys
import threading
from time import sleep
try:
    import thread
except ImportError:
    import _thread as thread

def quit_function():
        # print to stderr, unbuffered in Python 2.
        print('Parameter set rejected', file=sys.stderr)
        sys.stderr.flush() # Python 3 stderr is likely buffered.
        thread.interrupt_main() # raises KeyboardInterrupt
        

def exit_after(s):
    '''
    use as decorator to exit process if 
    function takes longer than s seconds
    '''
    def outer(fn):
        def inner(*args, **kwargs):
            timer = threading.Timer(s, quit_function, args=[])
            timer.start()
            try:
                result = fn(*args, **kwargs)
            finally:
                timer.cancel()
            return result
        return inner
    return outer

##Initialize timeout function
@exit_after(300)
def min_func(paramset):
    
    print(paramset)
    with open('paramset_rural.txt', 'a') as param:
        param.write('{params}\n'.format(params=paramset))

        
    try:    
        ##Extract median NRMSE value from model run
        print('started run')
        res_df = new_compare(rural,iters=25,
                             fus_fis=paramset[0],
                             no_ev=paramset[1],
                             rt_chg=paramset[2],
                             b_j=paramset[3],
                             d_j=paramset[4],
                             d_a=paramset[5],
                             ae_j=paramset[6],
                             ae_s=paramset[7],
                             ae_a=paramset[8],
                             t_p=paramset[9],
                             por=paramset[10],
                             fsr=paramset[11],
                             thresh=paramset[12],
                             fis_thresh=paramset[13],
                             lit_size=paramset[14],
                             dfe='off'
                            )
        print('finished run')
        smape = res_df.loc[res_df.sMAPE<=100]
        smape = smape.loc[:,'sMAPE'].dropna()

        total = len(smape)
        good = len(smape.loc[smape <= 15])
        perc = 1-(good/total)
        
        print('Percent of High Error Runs: {}'.format(perc))
        with open('percenterr_rural.txt', 'a') as percenterr:
            percenterr.write('{percents}\n'.format(percents=perc))
        
        
        return perc
    
    except:
        return 5

In [ ]:
##USE PYSWARM HERE
from pyswarm import pso

#x0 = [0.4,0.8,0.1,0.299,0.345,0.216,0.143,0.5,0.5,0.5,0.3,0.2,0.4,4,45,4]

lb = [0.1,.8,.05,.254,.278,.136,.034,.1,.1,.1,.15,.1,.2,4,38,3]
ub = [0.7,.95,.5,.344,.412,.296,.252,.8,.8,.8,.70,.6,.8,6,68,5]

xopt,fopt = pso(min_func,lb,ub,maxiter=150,debug=True,minfunc=0.01)

No constraints given.
[ 0.169971    0.90578026  0.38354683  0.34166802  0.34054933  0.17873813
  0.1973211   0.46382104  0.5945659   0.30462542  0.23093119  0.1438988
  0.65406211  5.61975214 62.27330106  3.52988653]
started run


/apps/software/standard/core/jupyter_conda/5.2.0-py3.6/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


[3.60467387e-01 8.86400128e-01 3.26991737e-01 3.09690592e-01
 3.51029973e-01 1.50971472e-01 3.86400522e-02 3.25602787e-01
 4.30839132e-01 4.56827118e-01 4.70910913e-01 2.04838740e-01
 3.73322177e-01 5.95994797e+00 5.70758148e+01 4.50259370e+00]
started run
[ 0.51691184  0.81226404  0.45969076  0.33843252  0.32098774  0.1429559
  0.14691054  0.24393064  0.27330132  0.59503124  0.56797175  0.3547972
  0.29050073  5.68512493 59.20529215  3.95222718]
started run
[ 0.22832813  0.94921941  0.44803327  0.25532856  0.35422731  0.20388978
  0.06543468  0.55982353  0.79637203  0.18899648  0.66849315  0.25012623
  0.70024834  5.00975325 57.66294684  3.22654432]
started run
[ 0.46021786  0.93536831  0.45936267  0.29315175  0.3393564   0.19834698
  0.08941093  0.70000587  0.49739188  0.39228183  0.62752494  0.4063903
  0.71658264  4.75980991 38.2061187   3.83550349]
started run
[2.96058141e-01 9.04994168e-01 1.26926823e-01 2.86640139e-01
 3.70760911e-01 2.35002946e-01 4.01548235e-02 5.72526331e-01


In [ ]:
xopt
with open('optimiz_rural.txt', 'a') as optimiz:
        optimiz.write('{params}\n'.format(params=xopt))